## AUTOMATIC CROP DISEASE DETECTION SYSTEM

#### Problem Statement
Most farmers are faced with problems like, low production as well as low profits due to disease attack on the plants. They are also faced with the problem of disease identification in plants as well as proper treatments for them. Currently, the farmers determine the type of disease manually. The errors might occur in order to determine the type of diseases. These farmers also must spend a lot of time to detect the type of disease and this reason they tend to waste time which they can channel in cultivation of more plants. 

#### Objectives
The objectives to achieve in this project are: 
i) To develop the prototype of disease detection system.
 ii) To detect the plant diseases by using image processing.
   iii) To apply image processing technique to analyze the pattern of plant disease.
      iv) To produce possible treatments to the plant diseases.




### Import relevant python packages

In [9]:
from tkinter import *
import tkinter as tk 
from tkinter import ttk 
import pandas as pd
from tkinter import filedialog
from PIL import ImageTk,Image
import tensorflow as tf
import numpy as np
import cv2
from tkinter import messagebox


### Load tensorflow model and tags
The downloaded zip file from Azure Custom Vision contains a model.pb and a labels.txt. These files represent the trained model and the classification labels. The first step is to load the model into your project.

In [10]:

graph_def = tf.compat.v1.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
filename = "model.pb"
labels_filename = "labels.txt"

# Import the TF graph
with tf.io.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

### Prepare an image for prediction
These are a few steps taken to prepare the image for prediction. These steps mimic the image manipulation performed during training:

In [11]:
#These are helper functions that are used to prepare the image for prediction.

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    image = image.convert('RGB')
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image 

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image   

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)


def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image
    
 # Return to default gui after prediction   
def clear():
    result_panel.destroy()
    treatment_panel.destroy()
    treatment.destroy()
    panel.destroy()
    
# Loads image from a file    
def openfilename(): 
    # open file dialog box to select image 
    # The dialogue box has a title "Open" 
    filename = filedialog.askopenfilename(title ='open') 
    return filename 
    
# Loads the image and perfoms classification. 
def classify_image():
    global panel,result_panel,treatment_panel,treatment 
    image_object = openfilename()
    
    # opens the image 
    image = Image.open(image_object) 
      
    # resize the image and apply a high-quality down sampling filter 
    img = image.resize((250, 250), Image.ANTIALIAS) 
  
    # PhotoImage class is used to add image to widgets, icons etc 
    img = ImageTk.PhotoImage(img) 
   
    # create a label 
    panel = Label(root, image=img) 
      
    # set the image as img  
    panel.image = img 
    panel.grid(row = 4 , column = 125)
    
    # Update orientation based on EXIF tags, if the file has orientation info.
    image = update_orientation(image)
    image = convert_to_opencv(image)
    image = resize_down_to_1600_max_dim(image)
    
    # We next get the largest center square
    h, w = image.shape[:2]
    min_dim = min(w,h)
    max_square_image = crop_center(image, min_dim, min_dim)
    
    # Resize that square down to 256x256
    augmented_image = resize_to_256_square(max_square_image)
    
    # Get the input size of the model
    with tf.compat.v1.Session() as sess:
        input_tensor_shape = sess.graph.get_tensor_by_name('Placeholder:0').shape.as_list()
    network_input_size = input_tensor_shape[1]
    
    # Crop the center for the specified network_input_Size
    augmented_image = crop_center(augmented_image, network_input_size, network_input_size) 

    # These names are part of the model and cannot be changed.
    output_layer = 'loss:0'
    input_node = 'Placeholder:0'
    
    with tf.compat.v1.Session() as sess:
        try:
            prob_tensor = sess.graph.get_tensor_by_name(output_layer)
            predictions = sess.run(prob_tensor, {input_node: [augmented_image]})
        except KeyError:
            print("Couldn't find classification output layer " + output_layer + '.')
            print('Verify this is a model exported from an Object Detection project.')
            exit(-1)
    
    highest_probability_index = np.argmax(predictions)
    
    # display treatments after image is classified.
    
    result_panel = Label(root, text="Name of disease: " + labels[highest_probability_index], font=("Copper Black", 18), fg='green')
    result_panel.grid(row=9, column=125)  
    if highest_probability_index == 0:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column =125 )
        treatment_panel = Label(root, text = " Treatment involves cutting away all infected branches and areas as well as bark into good wood." , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
       
    if highest_probability_index == 1:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column =125 )
        treatment_panel = Label(root, text = "All infected plants should be uprooted and destroyed to avoid disease build up. " , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
          
    if highest_probability_index == 2:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column =125 )
        treatment_panel = Label(root, text = " Existing galls can be removed with a sharp pruning knife " , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
          
    if highest_probability_index == 3:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column =125 )
        treatment_panel = Label(root, text = "Remove and destroy all fallen fruit and infected plants " , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
          
          
    if highest_probability_index == 4:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column =125 )
        treatment_panel = Label(root, text = "Remove old vegetable debris in the garden\n and do not plant new crops where host plants were once growing." , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
        
    if highest_probability_index == 5:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10, column =125 )
        treatment_panel = Label(root, text = " There are no cures for viral diseases such as mosaic once a plant is infected.\nAs a result,every effort should be made to prevent the disease from entering your garden" , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
          
    if highest_probability_index == 6:
        messagebox.showinfo("Error", "Image not identified")
        result_panel.destroy()
        treatment_panel.destroy()
        treatment.destroy()
      
    if highest_probability_index == 7:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "Standard fungicides are an effective way to manage powdery mildew disease on plants.\n Conventional fungicides should be applied on a regular basis for best results against the disease." , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
        
    if highest_probability_index == 8:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "Infected plants should be removed as soon as symptoms are noticed" , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125)
        
    if highest_probability_index == 9:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "To treat root rot, is recommended to replant the affected plant in fresh soil with\n good drainage to avoid standing water. " , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125) 
        
    if highest_probability_index == 10:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "Apply copper sprays or sulfur powders to prevent infection of susceptible plants. " , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125)
    
    if highest_probability_index == 11:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = " Treat with mercuric chloride to reduce the number of infected tubers over infected untreated seed." , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125)
    
    if highest_probability_index == 12:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "Control is difficult because once symptoms are observed,damage to the stem or roots\n is usually severe.\nA fungicide could also be mixed with soil prior to planting \n as suggested by themanufacturer." , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125)
        
    if highest_probability_index == 13:
        treatment = Label(root, text = "Treatment: ", fg = 'green',font=("Copper Black", 15))
        treatment.grid(row= 10 , column = 125 )
        treatment_panel = Label(root, text = "Remove stricken growth from the garden and sterilize pruning clippers between cuts" , font=("Copper Black", 12),fg = 'black')
        treatment_panel.grid(row = 11, column = 125)
            


### Displays Graphical User Interface

In [12]:
 root = tk.Tk()
root.geometry('300x300')
root.title('GREEN CLINIC')
#root['background']='white'
root.iconbitmap('tea-plant-leaf-icon.ico')
my_img = ImageTk.PhotoImage(Image.open("plant.png"))
can = 0.80
us = 0.85
name_app = Label(root, text = "CROP DISEASE DIAGNOSIS ", font=("Copper Black", 18),fg = 'green').grid(row = 2 , column = 125)

pic_label = Label(image= my_img).grid(row=1, column = 125)

space = Label(root, text = "                                 ", fg = 'green').grid(row = 0, column = 10)
spce = Label(root, text = "                                 ", fg = 'green').grid(row = 3, column = 125)
space1 = Label(root, text = "                                 ", fg = 'green').grid(row = 5, column = 20)
space2 = Label(root, text = "                                 ", fg = 'green').grid(row = 7, column = 30)
spacea = Label(root, text = "                                 ", fg = 'green').grid(row = 2, column = 40)
#spacea = Label(root, text = "                                 ", fg = 'green').grid(row = 9, column = 125)
spaceb = Label(root, text = "                                 ", fg = 'green').grid(row = 49, column = 50)
spaceb = Label(root, text = "                                 ", fg = 'green').grid(row = 8, column = 50)
spacec = Label(root, text = "                                 ", fg = 'green').grid(row = 19, column = 125)
space5 = Label(root, text = "                                 ", fg = 'green').grid(row = 199, column = 50)
upload = Button(root, text = " Upload image ", fg = 'black' , font=("Copper Black", 13),command = classify_image).grid(row =6,column = 125)
#results = Label(root, text = "Results are displayed here", fg = 'red',font=("fontslant", 10)).grid(row = 8,column = 126)

clear = Button(root, text = " Clear ", fg = 'black' , font=("Copper Black", 10), command= clear).grid(row = 7,column = 125)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-11-20ccc0bab154>", line 162, in classify_image
    treatment_panel.destroy()
  File "C:\Users\HP\anaconda3\lib\tkinter\__init__.py", line 2575, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
